<a href="https://colab.research.google.com/github/kathiravan1/100-Days-Of-ML-Code/blob/master/SVD_imbalance_toxic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('/content/sample_data/toxic.csv')

#target_count = df_train.label.value_counts()
#print('Class 0:', target_count[0])
#print('Class 1:', target_count[1])
#print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')
#target_count = df_train.groupby('label').size()
#target_count.plot(kind='bar', title='Count (label)');

In [3]:
df_train.head()

,comment,label
0,"Damn, this writing was pretty chaotic",0
1,"Yeah, and apparently a bunch of misogynistic v...",0
2,How the FUCK is this woman still an MP!!!???,0
3,Understand. Know you're right. At same time I ...,0
4,Surprized they didn't stop and rape some women,0


In [4]:
# Class count
#count_class_0, count_class_1 = df_train.label.value_counts()

# Divide by class
#df_class_0 = df_train[df_train['label'] == 0]
#df_class_1 = df_train[df_train['label'] == 1]

In [5]:
import nltk
import pandas as pd
import numpy as np
import plotly.express as plot
from wordcloud import WordCloud, STOPWORDS
from matplotlib import pyplot as mplot
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm.notebook import tqdm as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#starting essential code
nltk_STOPWORDS = set(stopwords.words("english"))
tqdm.pandas()

In [8]:
#df_class_0_under = df_class_0.sample(count_class_1)
#df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

#print('Random under-sampling:')
#print(df_test_under.label.value_counts())

#df_test_under.label.value_counts().plot(kind='bar', title='Count (label)');

In [9]:
df = df_train[df_train.comment.map(len) > 20]
df.shape

(13791, 2)

In [11]:
df["tokens"] = df_train.comment.progress_apply(word_tokenize)

  0%|          | 0/14000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
def removePunctuation(tokens)->list:
    return [t for t in tokens if t.isalpha()]

df["tokens"] = df.tokens.progress_apply(removePunctuation)

  0%|          | 0/13791 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
def removeStopwords(tokens) -> list:
    return [t for t in tokens if t not in nltk_STOPWORDS]

df["tokens"] = df.tokens.progress_apply(removeStopwords)

  0%|          | 0/13791 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
def removeSingleLenWords(tokens)->list:
    return [t for t in tokens if len(t) >= 2]
        

df["tokens"] = df.tokens.progress_apply(removeSingleLenWords)

  0%|          | 0/13791 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
def lematize(tokens, lematizer)->list:
    tokens = [lematizer.lemmatize(t, pos = "v") for t in tokens]
    return [lematizer.lemmatize(t, pos = "n") for t in tokens]
lemme = WordNetLemmatizer()
df["tokens"] = df.tokens.progress_apply(lambda tokens: lematize(tokens,lemme))

  0%|          | 0/13791 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
docs = df["tokens"].progress_apply(lambda tokens : ' '.join(tokens))

bagOfWords_model = vectorizer.fit(docs)
bagOfWords = bagOfWords_model.transform(docs)

  0%|          | 0/13791 [00:00<?, ?it/s]

In [18]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2000, n_iter=25, random_state=42)
svd_model = svd.fit(bagOfWords, df_train['label'])

In [19]:
data = svd_model.transform(bagOfWords)

In [20]:
(train_data, test_data, train_labels, test_labels) = train_test_split(data, df.label, test_size=0.3,random_state=563, shuffle=True)

In [21]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
NB_Model = clf.fit(train_data, train_labels)

In [22]:
print(classification_report(NB_Model.predict(test_data).round(), test_labels))

              precision    recall  f1-score   support

           0       0.71      0.82      0.76      2744
           1       0.48      0.33      0.39      1394

    accuracy                           0.65      4138
   macro avg       0.59      0.57      0.57      4138
weighted avg       0.63      0.65      0.63      4138



In [23]:
from sklearn.metrics import accuracy_score

In [24]:
from sklearn.svm import SVC
SVM = SVC(gamma='auto',probability=True,random_state=34)
SVM_model = SVM.fit(train_data,train_labels)

In [25]:
svm_predict = SVM_model.predict(test_data).round()
svm_accuracy = accuracy_score(svm_predict, test_labels)
print(classification_report(svm_predict, test_labels))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87      4138
           1       0.00      0.00      0.00         0

    accuracy                           0.77      4138
   macro avg       0.50      0.38      0.43      4138
weighted avg       1.00      0.77      0.87      4138



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.ensemble import RandomForestClassifier
randForestCalassifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=23)
rf_model = randForestCalassifier.fit(train_data,train_labels)

In [27]:
rf_predict = SVM_model.predict(test_data).round()
rf_accuracy = accuracy_score(rf_predict, test_labels)
print(classification_report(rf_predict, test_labels))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87      4138
           1       0.00      0.00      0.00         0

    accuracy                           0.77      4138
   macro avg       0.50      0.38      0.43      4138
weighted avg       1.00      0.77      0.87      4138



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs=-1, random_state=23)
lr_model = lr.fit(train_data,train_labels)

In [29]:
lr_predict = lr_model.predict(test_data).round()
lr_accuracy = accuracy_score(lr_predict, test_labels)
print(classification_report(lr_predict, test_labels))

              precision    recall  f1-score   support

           0       0.93      0.86      0.89      3437
           1       0.49      0.67      0.56       701

    accuracy                           0.83      4138
   macro avg       0.71      0.76      0.73      4138
weighted avg       0.85      0.83      0.84      4138



In [30]:
from sklearn.ensemble import VotingClassifier
VC = VotingClassifier(estimators=[('SVM', SVM), ('Random Forest', randForestCalassifier), ('Logistic Regression', lr)],weights=[svm_accuracy,rf_accuracy,lr_accuracy], voting='hard',n_jobs=-1)
VC_model = VC.fit(train_data,train_labels)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [31]:
vc_predict = VC_model.predict(test_data).round()
vc_accuracy = accuracy_score(vc_predict, test_labels)
print(classification_report(vc_predict, test_labels))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88      4020
           1       0.12      0.97      0.21       118

    accuracy                           0.79      4138
   macro avg       0.56      0.88      0.55      4138
weighted avg       0.97      0.79      0.86      4138



In [32]:
from sklearn.ensemble import StackingClassifier
SC = StackingClassifier(estimators=[('SVM', SVM), ('Random Forest', randForestCalassifier), ('Logistic Regression', lr)], final_estimator=lr,n_jobs=-1)
SC_model = SC.fit(train_data,train_labels)

In [33]:
sc_predict = SC_model.predict(test_data).round()
sc_accuracy = accuracy_score(sc_predict, test_labels)
print(classification_report(sc_predict, test_labels))

              precision    recall  f1-score   support

           0       0.96      0.85      0.90      3615
           1       0.43      0.78      0.55       523

    accuracy                           0.84      4138
   macro avg       0.70      0.82      0.73      4138
weighted avg       0.90      0.84      0.86      4138



In [35]:
import pickle
with open("models.bin","wb") as fp:
    pickle.dump({
        "bow":{"model":bagOfWords_model},
        "svd":{"model":svd_model},
        "lr":{"model":lr_model,"acc":lr_accuracy},
        "svm":{"model":SVM_model,"acc":svm_accuracy},
        "rf":{"model":rf_model,"acc":rf_accuracy},
        "vc":{"model":VC_model,"acc":vc_accuracy},
        "sc":{"model":SC_model,"acc":sc_accuracy}
    },fp)